
# Importing Libraries

In [ ]:
import cv2
import numpy as np
import os
import tensorflow as tf
import pandas as pd
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn import svm
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Flatten,InputLayer,Conv2D,MaxPooling2D,Dropout,MaxPool2D,Softmax,ReLU,GlobalAveragePooling2D
from tensorflow.keras.activations import softmax
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19

In [ ]:
# Normalization and adding noises

In [ ]:
def mat2gray(img):
    A = np.double(img)
    out = np.zeros(A.shape, np.double)
    normalized = cv2.normalize(A, out, 1.0, 0.0, cv2.NORM_MINMAX)
    return out
def random_noise(image, mode='gaussian', seed=None, clip=True, **kwargs):
    image = mat2gray(image)
    mode = mode.lower()
    if image.min() < 0:
        low_clip = -1
    else:
        low_clip = 0
    if seed is not None:
        np.random.seed(seed=seed)
    if mode == 'gaussian':
        noise = np.random.normal(kwargs['mean'], kwargs['var'] ** 0.5,image.shape)        
        out = image  + noise
    if clip:        
        out = np.clip(out, low_clip, 1.0)
    return out


Preprocessing of dataset with gabor filter and augumentation

In [ ]:
ksize =18  #Use size that makes sense to the image and fetaure size. Large may not be good. 
           #On the synthetic image it is clear how ksize affects imgae (try 5 and 50)
sigma = 1.5 #Large sigma on small features will fully miss the features. 
theta = 3*np.pi/4 #/4 shows horizontal 3/4 shows other horizontal. Try other contributions
lamda = 5  #1/4 works best for angled. 
gamma=1.5 #Value of 1 defines spherical. Calue close to 0 has high aspect ratio
        #Value of 1, spherical may not be ideal as it picks up features from other regions.
phi = 0 #Phase offset. I leave it to 0. (For hidden pic use 0.8)
kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, phi, ktype=cv2.CV_32F)
kernel_resized = cv2.resize(kernel, (400, 400))  # Resize image

img_data_array=[]
class_name=[]
#give path only till train folder
img_folder="C:\\Users\\luhar\\emotion\\emotion classifier\\ck+"
for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
            image_path= os.path.join(img_folder, dir1,  file)
            image= cv2.imread( image_path)
            image=cv2.resize(image,(126,126))
            
            ##------------------------------Apply Gabour filter here-----------------------------##
            image = cv2.filter2D(image, cv2.CV_8UC3, kernel)
            
            ##------------------------------Code for Gabour filter Complete here_------------------##
            image=np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            img1=imagenet_utils
            #img1 = random_noise(image,'gaussian', mean=0.1,var=0.1)
            img_data_array.append(img1)
            class_name.append(dir1)
            class_name.append(dir1)
label_encoder = preprocessing.LabelEncoder()
class_name= label_encoder.fit_transform(class_name)
img_data_array=np.array(img_data_array)
img_data_array[0].shape
x_train,x_test,y_train,y_test=train_test_split(img_data_array,class_name,test_size=0.1,shuffle=True)
print(x_train.shape)
print(x_test.shape)

Preprocessing of dataset without gabor filter

In [ ]:
img_data_array=[]
class_name=[]
#give path only till train folder
img_folder="C:\\Users\\luhar\\OneDrive\\Documents\\Code with ShiviSandy\\emotion_classifier\\ck+"
for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
            image_path= os.path.join(img_folder, dir1,  file)
            image= cv2.imread( image_path)
            image=cv2.resize(image,(126,126))
            image=np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            class_name.append(dir1)
label_encoder = preprocessing.LabelEncoder()
class_name= label_encoder.fit_transform(class_name)
img_data_array=np.array(img_data_array)
img_data_array[0].shape
x_train,x_test,y_train,y_test=train_test_split(img_data_array,class_name,test_size=0.1,shuffle=True)
print(x_train.shape)
print(x_test.shape)

# Preprocessing Training dataset

In [ ]:
plt.imshow(img_data_array[3])

# Add the different models here

## Custom Model No 1


In [ ]:
model=Sequential([
        Conv2D(32,kernel_size=(5,5),activation="relu",padding="SAME",input_shape=img_data_array[0].shape),
        Conv2D(64,kernel_size=(5,5),activation="relu",padding="SAME"),
        MaxPooling2D(pool_size=(2,2)),
        Conv2D(128,kernel_size=(3,3),activation="relu",padding="SAME"),
        Dropout(0.5),
        MaxPooling2D(pool_size=(2,2)),
        Flatten(),
        Dense(1024,activation="relu"),
        Dropout(0.5),
        Dense(512,activation="relu"),
        Dropout(0.5),
        Dense(7,activation="softmax")
    ])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history=model.fit(x_train,y_train,validation_split=0.1,epochs=30,shuffle=True)
frame = pd.DataFrame(history.history)
print(frame)

##Custom Model no 2

In [ ]:
model=Sequential([
    Conv2D(6,kernel_size=1,input_shape=img_data_array[0].shape),
    ReLU(),
    MaxPooling2D((2,2)),
    Conv2D(16,kernel_size=1),
    ReLU(),
    MaxPooling2D((2,2)),
    Conv2D(120,kernel_size=5),
    ReLU(),
    Dropout(0.3),
    Flatten(),
    Dense(84,activation='relu'),
    Dropout(0.3),
    Dense(7,activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history=model.fit(x_train,y_train,validation_split=0.1,epochs=30)
frame = pd.DataFrame(history.history)
print(frame)

In [ ]:
model=ResNet50(input_shape=img_data_array[0].shape)

In [ ]:
frame=frame.iloc[0:27]

In [ ]:
frame=frame.iloc[1:21]
frame

# Model Fitting


In [ ]:
frame1=frame.plot(y="loss",title="Epocs vs loss")
frame1.set(xlabel="Epocs",ylabel="loss")
frame1.set()

In [ ]:
frame.plot( y=["accuracy","val_accuracy"], figsize=(9, 8))
plt.title("Epoch vs accuracy")
plt.xlabel("# of Epoch")
plt.ylabel("Accuracy")
plt.show()

# Evaluating model

In [ ]:
loss,acc=model.evaluate(x_test,y_test,verbose=2)
print(loss)
print(acc)